In [2]:
import numpy as np
import pandas as pd
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

## Depression Data 

In [2]:
depression_train = pd.read_csv('train2Data.csv')
depression_train

,Tweets,Label
0,damn taking this personality quiz and realizin...,1
1,remember experiencing as kid what now realize ...,1
2,damn louis really did pull me out of my depres...,1
3,my depression is really kicking my ass right n...,1
4,feel like my night is going bad family calling...,1
...,...,...
30995,dnc puppet is claiming she wants to put god am...,0
30996,as soon as my election job is over ll be back ...,0
30997,the moral of the story is you gotta get it fir...,0
30998,got through my hardest week this semester and ...,0


In [5]:
depression_train.describe()

,Label
count,31000.000000
mean,0.516129
std,0.499748
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [5]:
X_train.values[:10]

array([['damn taking this personality quiz and realizing have severe depression'],
       ['remember experiencing as kid what now realize was clinical depression and at the time not knowing how to identify what was feeling other than just don want anything complete and utter lack of desire to exist or do damn thing'],
       ['damn louis really did pull me out of my depression painting for the first time in months'],
       ['my depression is really kicking my ass right now so damn tired'],
       ['feel like my night is going bad family calling me grumpy and whiny like so damn sorry for having depression'],
       ['im really trying to embrace it my depression needs to take vacation for scorpio season damn'],
       ['please my mom put me on strict no carb diet know im fatass but damn bitch wants to eat her depression away'],
       ['being on my medication has helped me so much like literally have the worst memory have adhd depression anxiety and ve had ptsd so all this has made my m

In [102]:
X_train = depression_train.iloc[:,:-1]
y_train = depression_train.iloc[:,-1]
print(X_train.values)
print(y_train)

[['damn taking this personality quiz and realizing have severe depression']
 ['remember experiencing as kid what now realize was clinical depression and at the time not knowing how to identify what was feeling other than just don want anything complete and utter lack of desire to exist or do damn thing']
 ['damn louis really did pull me out of my depression painting for the first time in months']
 ...
 ['the moral of the story is you gotta get it first like child certificate when woman gives birth it would be on record to help blacks but they only interested in keeping us back making promises after he gets elected is like car loan you wanted amp got rejected']
 ['got through my hardest week this semester and feel confident on the rest of the year']
 ['lately ve been trying to make an effort to take note of positives and good all there are lot of good people in the world']]
0        1
1        1
2        1
3        1
4        1
        ..
30995    0
30996    0
30997    0
30998    0
3099

In [103]:
y_train_dict = Counter(y_train)
total = sum(y_train_dict.values())
print("Balanced Checking: ", Counter(y_train))
print("Label 0: %.2f%%" % (y_train_dict.get(0)*100 / total))
print("Label 1: %.2f%%" % (y_train_dict.get(1)*100 / total))

Balanced Checking:  Counter({1: 16000, 0: 15000})
Label 0: 48.39%
Label 1: 51.61%


In [107]:
depression_test = pd.read_csv('test2Data.csv')
depression_test

,Tweets,Label
0,just know what ur going through battle depress...,1
1,wow depression and anxiety really tryna come f...,1
2,you aint making joke about someones depression...,1
3,why do insta meme pages post videos of naked w...,1
4,cant handle school anymore but hate in person ...,1
...,...,...
30995,the only way they can attract good dicks is to...,0
30996,gotta say like you think we have the same atti...,0
30997,many product companies goes from right to left...,0
30998,just casually playing animal crossing while wa...,0


In [111]:
X_test = depression_test.iloc[:,:-1]
y_test = depression_test.iloc[:,-1]
print(X_test)
print(y_test)

                                                  Tweets
0      just know what ur going through battle depress...
1      wow depression and anxiety really tryna come f...
2      you aint making joke about someones depression...
3      why do insta meme pages post videos of naked w...
4      cant handle school anymore but hate in person ...
...                                                  ...
30995  the only way they can attract good dicks is to...
30996  gotta say like you think we have the same atti...
30997  many product companies goes from right to left...
30998  just casually playing animal crossing while wa...
30999  in kushner speak look wasn born with silver sp...

[31000 rows x 1 columns]
0        1
1        1
2        1
3        1
4        1
        ..
30995    0
30996    0
30997    0
30998    0
30999    0
Name:  Label, Length: 31000, dtype: int64


In [112]:
y_test_dict = Counter(y_test)
total = sum(y_test_dict.values())
print("Balanced Checking: ", Counter(y_test))
print("Label 0: %.2f%%" % (y_test_dict.get(0)*100 / total))
print("Label 1: %.2f%%" % (y_test_dict.get(1)*100 / total))

Balanced Checking:  Counter({1: 16000, 0: 15000})
Label 0: 48.39%
Label 1: 51.61%


In [105]:
import nltk
# Uncomment to download "stopwords"
#nltk.download("stopwords")
from nltk.corpus import stopwords

def text_preprocessing(s):
    """
    - Lowercase the sentence
    - Change "'t" to "not"
    - Remove "@name"
    - Isolate and remove punctuations except "?"
    - Remove other special characters
    - Remove stop words except "not" and "can"
    - Remove trailing whitespace
    """
    s = s.lower()
    # Change 't to 'not'
    s = re.sub(r"\'t", " not", s)
    # Remove @name
    s = re.sub(r'(@.*?)[\s]', ' ', s)
    # Isolate and remove punctuations except '?'
    s = re.sub(r'([\'\"\.\(\)\!\?\\\/\,])', r' \1 ', s)
    s = re.sub(r'[^\w\s\?]', ' ', s)
    # Remove some special characters
    s = re.sub(r'([\;\:\|•«\n])', ' ', s)
    # Remove stopwords except 'not' and 'can'
    s = " ".join([word for word in s.split()
                  if word not in stopwords.words('english')
                  or word in ['not', 'can']])
    # Remove trailing whitespace
    s = re.sub(r'\s+', ' ', s).strip()
    
    return s

In [108]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer

# Preprocess text
X_train_preprocessed = np.array([text_preprocessing(text) for text in depression_train['Tweets'].values])
X_test_preprocessed = np.array([text_preprocessing(text) for text in depression_test['Tweets'].values])

# Calculate TF-IDF
tf_idf = TfidfVectorizer(ngram_range=(1, 3), smooth_idf=False)

X_train_tfidf = tf_idf.fit_transform(X_train_preprocessed)
X_test_tfidf = tf_idf.transform(X_test_preprocessed)

In [109]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB().fit(X_train_tfidf, depression_train[' Label'].values)

In [110]:
y_test = depression_test[' Label'].values

In [111]:
from sklearn.metrics import  classification_report, confusion_matrix, accuracy_score, f1_score
labels_pred = nb.predict(X_test_tfidf)
#labels_pred = np.round(labels_pred.flatten())
accuracy = accuracy_score(y_test, labels_pred)
f1 = f1_score(y_test, labels_pred, average='weighted')
precision = precision_score(y_test, labels_pred, average='weighted')
recall = recall_score(y_test, labels_pred, average='weighted')

print("Accuracy: %.2f%%" % (accuracy*100))
print("F1 Score: %.2f" % (f1*100))
print("Precision: %.2f" % (precision*100))
print("Recall: %.2f" % (recall*100))


print(classification_report(y_test, labels_pred))

Accuracy: 91.18%
F1 Score: 91.09
Precision: 92.44
Recall: 91.18
              precision    recall  f1-score   support

           0       1.00      0.82      0.90     15000
           1       0.85      1.00      0.92     16000

    accuracy                           0.91     31000
   macro avg       0.93      0.91      0.91     31000
weighted avg       0.92      0.91      0.91     31000



In [112]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=42, max_iter=300).fit(X_train_tfidf, depression_train[' Label'].values)

In [113]:
labels_pred2 = clf.predict(X_test_tfidf)
#labels_pred = np.round(labels_pred.flatten())
accuracy_2 = accuracy_score(y_test, labels_pred2)
f1_2 = f1_score(y_test, labels_pred2, average='weighted')
precision_2 = precision_score(y_test, labels_pred2, average='weighted')
recall_2 = recall_score(y_test, labels_pred2, average='weighted')

print("Accuracy: %.2f%%" % (accuracy_2*100))
print("F1 Score: %.2f" % (f1_2*100))
print("Precision: %.2f" % (precision_2*100))
print("Recall: %.2f" % (recall_2*100))

print(classification_report(y_test, labels_pred2))

Accuracy: 98.35%
F1 Score: 98.34
Precision: 98.40
Recall: 98.35
              precision    recall  f1-score   support

           0       1.00      0.97      0.98     15000
           1       0.97      1.00      0.98     16000

    accuracy                           0.98     31000
   macro avg       0.98      0.98      0.98     31000
weighted avg       0.98      0.98      0.98     31000



In [114]:
from sklearn.ensemble import RandomForestClassifier
randomforest = RandomForestClassifier(n_estimators = 100, random_state = 24)
randomforest.fit(X_train_tfidf, depression_train[' Label'].values)

RandomForestClassifier(random_state=24)

In [115]:
labels_pred3 = randomforest.predict(X_test_tfidf)
#labels_pred = np.round(labels_pred.flatten())
accuracy_3 = accuracy_score(y_test, labels_pred3)
f1_3 = f1_score(y_test, labels_pred3, average='weighted')
precision_3 = precision_score(y_test, labels_pred3, average='weighted')
recall_3 = recall_score(y_test, labels_pred3, average='weighted')

print("Accuracy: %.2f%%" % (accuracy_3*100))
print("F1 Score: %.2f" % (f1_3*100))
print("Precision: %.2f" % (precision_3*100))
print("Recall: %.2f" % (recall_3*100))

print(classification_report(y_test, labels_pred3))

Accuracy: 99.40%
F1 Score: 99.40
Precision: 99.40
Recall: 99.40
              precision    recall  f1-score   support

           0       1.00      0.99      0.99     15000
           1       0.99      1.00      0.99     16000

    accuracy                           0.99     31000
   macro avg       0.99      0.99      0.99     31000
weighted avg       0.99      0.99      0.99     31000



In [116]:
from sklearn.svm import SVC
SVM = SVC(kernel='linear', random_state=42).fit(X_train_tfidf, depression_train[' Label'].values)

In [117]:
labels_pred4 = SVM.predict(X_test_tfidf)
#labels_pred = np.round(labels_pred.flatten())
accuracy_4 = accuracy_score(y_test, labels_pred4)
f1_4 = f1_score(y_test, labels_pred4, average='weighted')
precision_4 = precision_score(y_test, labels_pred4, average='weighted')
recall_4 = recall_score(y_test, labels_pred4, average='weighted')

print("Accuracy: %.2f%%" % (accuracy_4*100))
print("F1 Score: %.2f" % (f1_4*100))
print("Precision: %.2f" % (precision_4*100))
print("Recall: %.2f" % (recall_4*100))

print(classification_report(y_test, labels_pred4))

Accuracy: 98.98%
F1 Score: 98.98
Precision: 99.00
Recall: 98.98
              precision    recall  f1-score   support

           0       1.00      0.98      0.99     15000
           1       0.98      1.00      0.99     16000

    accuracy                           0.99     31000
   macro avg       0.99      0.99      0.99     31000
weighted avg       0.99      0.99      0.99     31000



## Emotion Data

In [76]:
df_train = pd.read_csv('../Dataset/emotion/train.txt', header =None, sep =';', names = ['Input','Sentiment'], encoding='utf-8')
df_train

,Input,Sentiment
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger
...,...,...
15995,i just had a very brief time in the beanbag an...,sadness
15996,i am now turning and i feel pathetic that i am...,sadness
15997,i feel strong and good overall,joy
15998,i feel like this was such a rude comment and i...,anger


In [77]:
X_train = df_train.iloc[:,:-1]
y_train = df_train.iloc[:,-1]
print(X_train)
print(y_train)

                                                   Input
0                                i didnt feel humiliated
1      i can go from feeling so hopeless to so damned...
2       im grabbing a minute to post i feel greedy wrong
3      i am ever feeling nostalgic about the fireplac...
4                                   i am feeling grouchy
...                                                  ...
15995  i just had a very brief time in the beanbag an...
15996  i am now turning and i feel pathetic that i am...
15997                     i feel strong and good overall
15998  i feel like this was such a rude comment and i...
15999  i know a lot but i feel so stupid because i ca...

[16000 rows x 1 columns]
0        sadness
1        sadness
2          anger
3           love
4          anger
          ...   
15995    sadness
15996    sadness
15997        joy
15998      anger
15999    sadness
Name: Sentiment, Length: 16000, dtype: object


In [78]:
y_train_dict = Counter(y_train)
total = sum(y_train_dict.values())
print("Balanced Checking: ", Counter(y_train))
print("Joy: %.2f%%" % (y_train_dict.get('joy')*100 / total))
print("Sadness: %.2f%%" % (y_train_dict.get('sadness')*100 / total))
print("Anger: %.2f%%" % (y_train_dict.get('anger')*100 / total))
print("Fear: %.2f%%" % (y_train_dict.get('fear')*100 / total))
print("Love: %.2f%%" % (y_train_dict.get('love')*100 / total))
print("Surprise: %.2f%%" % (y_train_dict.get('surprise')*100 / total))

Balanced Checking:  Counter({'joy': 5362, 'sadness': 4666, 'anger': 2159, 'fear': 1937, 'love': 1304, 'surprise': 572})
Joy: 33.51%
Sadness: 29.16%
Anger: 13.49%
Fear: 12.11%
Love: 8.15%
Surprise: 3.58%


In [79]:
df_test = pd.read_csv('../Dataset/emotion/test.txt', header =None, sep =';', names = ['Input','Sentiment'], encoding='utf-8')
df_test

,Input,Sentiment
0,im feeling rather rotten so im not very ambiti...,sadness
1,im updating my blog because i feel shitty,sadness
2,i never make her separate from me because i do...,sadness
3,i left with my bouquet of red and yellow tulip...,joy
4,i was feeling a little vain when i did this one,sadness
...,...,...
1995,i just keep feeling like someone is being unki...,anger
1996,im feeling a little cranky negative after this...,anger
1997,i feel that i am useful to my people and that ...,joy
1998,im feeling more comfortable with derby i feel ...,joy


In [80]:
X_test = df_test.iloc[:,:-1]
y_test = df_test.iloc[:,-1]
print(X_test)
print(y_test)

                                                  Input
0     im feeling rather rotten so im not very ambiti...
1             im updating my blog because i feel shitty
2     i never make her separate from me because i do...
3     i left with my bouquet of red and yellow tulip...
4       i was feeling a little vain when i did this one
...                                                 ...
1995  i just keep feeling like someone is being unki...
1996  im feeling a little cranky negative after this...
1997  i feel that i am useful to my people and that ...
1998  im feeling more comfortable with derby i feel ...
1999  i feel all weird when i have to meet w people ...

[2000 rows x 1 columns]
0       sadness
1       sadness
2       sadness
3           joy
4       sadness
         ...   
1995      anger
1996      anger
1997        joy
1998        joy
1999       fear
Name: Sentiment, Length: 2000, dtype: object


In [81]:
y_test_dict = Counter(y_test)
total = sum(y_test_dict.values())
print("Balanced Checking: ", Counter(y_test))
print("Joy: %.2f%%" % (y_test_dict.get('joy')*100 / total))
print("Sadness: %.2f%%" % (y_test_dict.get('sadness')*100 / total))
print("Anger: %.2f%%" % (y_test_dict.get('anger')*100 / total))
print("Fear: %.2f%%" % (y_test_dict.get('fear')*100 / total))
print("Love: %.2f%%" % (y_test_dict.get('love')*100 / total))
print("Surprise: %.2f%%" % (y_test_dict.get('surprise')*100 / total))

Balanced Checking:  Counter({'joy': 695, 'sadness': 581, 'anger': 275, 'fear': 224, 'love': 159, 'surprise': 66})
Joy: 34.75%
Sadness: 29.05%
Anger: 13.75%
Fear: 11.20%
Love: 7.95%
Surprise: 3.30%


In [82]:
df_val = pd.read_csv('../Dataset/emotion/val.txt', header =None, sep =';', names = ['Input','Sentiment'], encoding='utf-8')
df_val

,Input,Sentiment
0,im feeling quite sad and sorry for myself but ...,sadness
1,i feel like i am still looking at a blank canv...,sadness
2,i feel like a faithful servant,love
3,i am just feeling cranky and blue,anger
4,i can have for a treat or if i am feeling festive,joy
...,...,...
1995,im having ssa examination tomorrow in the morn...,sadness
1996,i constantly worry about their fight against n...,joy
1997,i feel its important to share this info for th...,joy
1998,i truly feel that if you are passionate enough...,joy


In [83]:
X_val = df_val.iloc[:,:-1]
y_val = df_val.iloc[:,-1]
print(X_val)
print(y_val)

                                                  Input
0     im feeling quite sad and sorry for myself but ...
1     i feel like i am still looking at a blank canv...
2                        i feel like a faithful servant
3                     i am just feeling cranky and blue
4     i can have for a treat or if i am feeling festive
...                                                 ...
1995  im having ssa examination tomorrow in the morn...
1996  i constantly worry about their fight against n...
1997  i feel its important to share this info for th...
1998  i truly feel that if you are passionate enough...
1999  i feel like i just wanna buy any cute make up ...

[2000 rows x 1 columns]
0       sadness
1       sadness
2          love
3         anger
4           joy
         ...   
1995    sadness
1996        joy
1997        joy
1998        joy
1999        joy
Name: Sentiment, Length: 2000, dtype: object


In [84]:
y_val_dict = Counter(y_val)
total = sum(y_val_dict.values())
print("Balanced Checking: ", Counter(y_val))
print("Joy: %.2f%%" % (y_val_dict.get('joy')*100 / total))
print("Sadness: %.2f%%" % (y_val_dict.get('sadness')*100 / total))
print("Anger: %.2f%%" % (y_val_dict.get('anger')*100 / total))
print("Fear: %.2f%%" % (y_val_dict.get('fear')*100 / total))
print("Love: %.2f%%" % (y_val_dict.get('love')*100 / total))
print("Surprise: %.2f%%" % (y_val_dict.get('surprise')*100 / total))

Balanced Checking:  Counter({'joy': 704, 'sadness': 550, 'anger': 275, 'fear': 212, 'love': 178, 'surprise': 81})
Joy: 35.20%
Sadness: 27.50%
Anger: 13.75%
Fear: 10.60%
Love: 8.90%
Surprise: 4.05%


In [85]:
import nltk
# Uncomment to download "stopwords"
#nltk.download("stopwords")
from nltk.corpus import stopwords

def text_preprocessing(s):
    """
    - Lowercase the sentence
    - Change "'t" to "not"
    - Remove "@name"
    - Isolate and remove punctuations except "?"
    - Remove other special characters
    - Remove stop words except "not" and "can"
    - Remove trailing whitespace
    """
    s = s.lower()
    # Change 't to 'not'
    s = re.sub(r"\'t", " not", s)
    # Remove @name
    s = re.sub(r'(@.*?)[\s]', ' ', s)
    # Isolate and remove punctuations except '?'
    s = re.sub(r'([\'\"\.\(\)\!\?\\\/\,])', r' \1 ', s)
    s = re.sub(r'[^\w\s\?]', ' ', s)
    # Remove some special characters
    s = re.sub(r'([\;\:\|•«\n])', ' ', s)
    # Remove stopwords except 'not' and 'can'
    s = " ".join([word for word in s.split()
                  if word not in stopwords.words('english')
                  or word in ['not', 'can']])
    # Remove trailing whitespace
    s = re.sub(r'\s+', ' ', s).strip()
    
    return s

In [86]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer

# Preprocess text
X_train_preprocessed = np.array([text_preprocessing(text) for text in X_train['Input'].values])
X_val_preprocessed = np.array([text_preprocessing(text) for text in X_val['Input'].values])

# Calculate TF-IDF
tf_idf = TfidfVectorizer(ngram_range=(1, 3), smooth_idf=False)

X_train_tfidf = tf_idf.fit_transform(X_train_preprocessed)
X_val_tfidf = tf_idf.transform(X_val_preprocessed)

In [87]:
# define undersampling strategy
undersample = RandomUnderSampler(random_state=0)

# fit and apply the transform
X_train_under, y_train_under = undersample.fit_resample(X_train_tfidf, y_train)

# summarize class distribution
print("After undersampling: ", Counter(y_train_under))

After undersampling:  Counter({'anger': 572, 'fear': 572, 'joy': 572, 'love': 572, 'sadness': 572, 'surprise': 572})


In [88]:
# define oversampling strategy
SMOTE = SMOTE(random_state=42)

# fit and apply the transform
X_train_SMOTE, y_train_SMOTE = SMOTE.fit_resample(X_train_tfidf, y_train)

# summarize class distribution
print("After oversampling: ", Counter(y_train_SMOTE))

After oversampling:  Counter({'sadness': 5362, 'anger': 5362, 'love': 5362, 'surprise': 5362, 'fear': 5362, 'joy': 5362})


In [89]:
encoding = { 'anger': 0,
    'fear': 1,
    'joy': 2,
    'love': 3,
    'sadness': 4,
    'surprise': 5
}

In [90]:
y_train_SMOTE_encode = [encoding[key] for key in y_train_SMOTE.values]
y_val = [encoding[key] for key in df_val['Sentiment'].values]
y_test = [encoding[key] for key in df_test['Sentiment'].values]

In [91]:
X_test_preprocessed = np.array([text_preprocessing(text) for text in X_test['Input'].values])
X_test_tfidf = tf_idf.transform(X_test_preprocessed)

In [92]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB().fit(X_train_SMOTE, y_train_SMOTE_encode)

In [94]:
from sklearn.metrics import  classification_report, confusion_matrix, accuracy_score, f1_score
labels_pred = nb.predict(X_test_tfidf)
#labels_pred = np.round(labels_pred.flatten())
accuracy = accuracy_score(y_test, labels_pred)
f1 = f1_score(y_test, labels_pred, average='weighted')
precision = precision_score(y_test, labels_pred, average='weighted')
recall = recall_score(y_test, labels_pred, average='weighted')

print("Accuracy: %.2f%%" % (accuracy*100))
print("F1 Score: %.2f" % (f1*100))
print("Precision: %.2f" % (precision*100))
print("Recall: %.2f" % (recall*100))


print(classification_report(y_test, labels_pred))

Accuracy: 84.10%
F1 Score: 84.63
Precision: 85.97
Recall: 84.10
              precision    recall  f1-score   support

           0       0.83      0.82      0.82       275
           1       0.83      0.78      0.81       224
           2       0.91      0.83      0.87       695
           3       0.61      0.85      0.71       159
           4       0.93      0.88      0.91       581
           5       0.50      0.85      0.63        66

    accuracy                           0.84      2000
   macro avg       0.77      0.84      0.79      2000
weighted avg       0.86      0.84      0.85      2000



In [62]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, labels_pred)

array([[226,  10,  13,   5,  17,   4],
       [  8, 175,   8,   4,  10,  19],
       [ 15,   8, 576,  69,   6,  21],
       [  3,   1,  14, 135,   4,   2],
       [ 20,  12,  17,   8, 514,  10],
       [  1,   4,   3,   1,   1,  56]])

In [63]:
import pickle
with open('./model/NB_Emotion.h5', 'wb') as model:
    pickle.dump(nb, model)

In [95]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=42, max_iter=300).fit(X_train_SMOTE, y_train_SMOTE_encode)

In [96]:
labels_pred2 = clf.predict(X_test_tfidf)
#labels_pred = np.round(labels_pred.flatten())
accuracy_2 = accuracy_score(y_test, labels_pred2)
f1_2 = f1_score(y_test, labels_pred2, average='weighted')
precision_2 = precision_score(y_test, labels_pred2, average='weighted')
recall_2 = recall_score(y_test, labels_pred2, average='weighted')

print("Accuracy: %.2f%%" % (accuracy_2*100))
print("F1 Score: %.2f" % (f1_2*100))
print("Precision: %.2f" % (precision_2*100))
print("Recall: %.2f" % (recall_2*100))

print(classification_report(y_test, labels_pred2))

Accuracy: 87.85%
F1 Score: 88.01
Precision: 88.43
Recall: 87.85
              precision    recall  f1-score   support

           0       0.87      0.87      0.87       275
           1       0.87      0.80      0.84       224
           2       0.91      0.89      0.90       695
           3       0.71      0.87      0.78       159
           4       0.94      0.91      0.92       581
           5       0.64      0.82      0.72        66

    accuracy                           0.88      2000
   macro avg       0.82      0.86      0.84      2000
weighted avg       0.88      0.88      0.88      2000



In [66]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, labels_pred2)

array([[238,   8,  14,   2,  13,   0],
       [  8, 180,   7,   1,  12,  16],
       [  4,   4, 619,  50,   7,  11],
       [  2,   0,  16, 138,   3,   0],
       [ 21,   7,  19,   3, 528,   3],
       [  0,   7,   4,   0,   1,  54]])

In [67]:
import pickle
with open('./model/LogReg_Emotion.h5', 'wb') as model:
    pickle.dump(clf, model)

In [97]:
from sklearn.ensemble import RandomForestClassifier
randomforest = RandomForestClassifier(n_estimators = 100, random_state = 24)
randomforest.fit(X_train_SMOTE, y_train_SMOTE_encode)

RandomForestClassifier(random_state=24)

In [98]:
labels_pred3 = randomforest.predict(X_test_tfidf)
#labels_pred = np.round(labels_pred.flatten())
accuracy_3 = accuracy_score(y_test, labels_pred3)
f1_3 = f1_score(y_test, labels_pred3, average='weighted')
precision_3 = precision_score(y_test, labels_pred3, average='weighted')
recall_3 = recall_score(y_test, labels_pred3, average='weighted')

print("Accuracy: %.2f%%" % (accuracy_3*100))
print("F1 Score: %.2f" % (f1_3*100))
print("Precision: %.2f" % (precision_3*100))
print("Recall: %.2f" % (recall_3*100))

print(classification_report(y_test, labels_pred3))

Accuracy: 88.10%
F1 Score: 88.01
Precision: 88.02
Recall: 88.10
              precision    recall  f1-score   support

           0       0.87      0.91      0.89       275
           1       0.91      0.82      0.86       224
           2       0.89      0.92      0.90       695
           3       0.82      0.75      0.78       159
           4       0.91      0.91      0.91       581
           5       0.67      0.61      0.63        66

    accuracy                           0.88      2000
   macro avg       0.84      0.82      0.83      2000
weighted avg       0.88      0.88      0.88      2000



In [70]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, labels_pred3)

array([[249,   0,  12,   0,  14,   0],
       [ 10, 184,   5,   0,  12,  13],
       [  6,   3, 638,  25,  19,   4],
       [  3,   0,  31, 120,   4,   1],
       [ 19,   4,  23,   2, 531,   2],
       [  0,  12,  11,   0,   3,  40]])

In [71]:
import pickle
with open('./model/RF_Emotion.h5', 'wb') as model:
    pickle.dump(randomforest, model)

In [99]:
from sklearn.svm import SVC
SVM = SVC(kernel='linear', random_state=42).fit(X_train_SMOTE, y_train_SMOTE_encode)

In [100]:
labels_pred4 = SVM.predict(X_test_tfidf)
#labels_pred = np.round(labels_pred.flatten())
accuracy_4 = accuracy_score(y_test, labels_pred4)
f1_4 = f1_score(y_test, labels_pred4, average='weighted')
precision_4 = precision_score(y_test, labels_pred4, average='weighted')
recall_4 = recall_score(y_test, labels_pred4, average='weighted')

print("Accuracy: %.2f%%" % (accuracy_4*100))
print("F1 Score: %.2f" % (f1_4*100))
print("Precision: %.2f" % (precision_4*100))
print("Recall: %.2f" % (recall_4*100))

print(classification_report(y_test, labels_pred4))

Accuracy: 88.65%
F1 Score: 88.73
Precision: 88.87
Recall: 88.65
              precision    recall  f1-score   support

           0       0.90      0.88      0.89       275
           1       0.86      0.82      0.84       224
           2       0.92      0.91      0.91       695
           3       0.74      0.82      0.78       159
           4       0.93      0.93      0.93       581
           5       0.63      0.70      0.66        66

    accuracy                           0.89      2000
   macro avg       0.83      0.84      0.84      2000
weighted avg       0.89      0.89      0.89      2000



In [76]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, labels_pred4)

array([[242,   6,  10,   1,  16,   0],
       [  8, 184,   5,   2,  12,  13],
       [  4,   2, 632,  39,   8,  10],
       [  0,   0,  24, 131,   3,   1],
       [ 16,   9,  12,   3, 538,   3],
       [  0,  13,   5,   0,   2,  46]])

In [77]:
import pickle
with open('./model/SVM_Emotion.h5', 'wb') as model:
    pickle.dump(SVM, model)

## Sentiment Data

In [6]:
DATASET_COLUMNS = ["target", "ids", "date", "flag", "user", "text"]
DATASET_ENCODING = "ISO-8859-1"

df = pd.read_csv('../Dataset/sentiment_tweets/training.1600000.processed.noemoticon.csv', encoding=DATASET_ENCODING, names=DATASET_COLUMNS)
df

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [7]:
df['target'] = df['target'].replace(4, 1)

In [8]:
X_train = df['text']
y_train = df['target']
print(X_train.values)
print(y_train)

["@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"
 "is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!"
 '@Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds'
 ... 'Are you ready for your MoJo Makeover? Ask me for details '
 'Happy 38th Birthday to my boo of alll time!!! Tupac Amaru Shakur '
 'happy #charitytuesday @theNSPCC @SparksCharity @SpeakingUpH4H ']
0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [9]:
y_train_dict = Counter(y_train)
total = sum(y_train_dict.values())
print("Balanced Checking: ", Counter(y_train))
print("Label 0: %.2f%%" % (y_train_dict.get(0)*100 / total))
print("Label 1: %.2f%%" % (y_train_dict.get(1)*100 / total))

Balanced Checking:  Counter({0: 800000, 1: 800000})
Label 0: 50.00%
Label 1: 50.00%


In [46]:
df_0 = df[df['target'] == 0]
df_1 = df[df['target'] == 1]
df_new = pd.concat([df_0[:25000], df_1[:25000]])

In [47]:
df_new

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
824995,1,1556381057,Sat Apr 18 22:37:08 PDT 2009,NO_QUERY,ethanattack,Yup. Going to sleep. Night guys.
824996,1,1556381076,Sat Apr 18 22:37:06 PDT 2009,NO_QUERY,GinaLaGuardia,"@MsUnitedStates Thank you so, so, so, so much ..."
824997,1,1556381131,Sat Apr 18 22:37:09 PDT 2009,NO_QUERY,CarolCalazans,@tommcfly Hey Tom How are you? http://twitpic....
824998,1,1556381147,Sat Apr 18 22:37:07 PDT 2009,NO_QUERY,KKVegas,Teasing and Mack 1-0 are taking Vegas by storm...


In [48]:
# 0: NEGATIVE
# 1: POSITIVE

print(Counter(df_new['target']))

Counter({0: 25000, 1: 25000})


In [49]:
import nltk
# Uncomment to download "stopwords"
#nltk.download("stopwords")
from nltk.corpus import stopwords

def text_preprocessing(s):
    """
    - Lowercase the sentence
    - Change "'t" to "not"
    - Remove "@name"
    - Isolate and remove punctuations except "?"
    - Remove other special characters
    - Remove stop words except "not" and "can"
    - Remove trailing whitespace
    """
    s = s.lower()
    # Change 't to 'not'
    s = re.sub(r"\'t", " not", s)
    # Remove @name
    s = re.sub(r'(@.*?)[\s]', ' ', s)
    s = re.sub(r'#(\w+)', ' ', s)
    # Isolate and remove punctuations except '?'
    s = re.sub(r'([\'\"\.\(\)\!\?\\\/\,])', r' \1 ', s)
    s = re.sub(r'[^\w\s\?]', ' ', s)
    # Remove some special characters
    s = re.sub(r'([\;\:\|•«\n])', ' ', s)
    # Remove stopwords except 'not' and 'can'
    s = " ".join([word for word in s.split()
                  if word not in stopwords.words('english')
                  or word in ['not', 'can']])
    # Remove trailing whitespace
    s = re.sub(r'\s+', ' ', s).strip()
    
    return s

In [50]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df_new, test_size=0.2, random_state=42)

In [51]:
df_train

,target,ids,date,flag,user,text
814087,1,1550708726,Sat Apr 18 07:01:04 PDT 2009,NO_QUERY,MissMIsanchez,@anterazor teacup yorkie
805893,1,1468728004,Tue Apr 07 03:24:24 PDT 2009,NO_QUERY,rahnocerous,@xCarCrashHearts on friday till monday so plan...
820278,1,1553676855,Sat Apr 18 14:54:52 PDT 2009,NO_QUERY,alejandradd,@nerdist He's doing great. He read almost 1000...
16398,0,1555901859,Sat Apr 18 21:01:41 PDT 2009,NO_QUERY,lovesickpause,"When did I lose my motivation, my momentum?"
13653,0,1553336123,Sat Apr 18 13:59:45 PDT 2009,NO_QUERY,JenniferBoydon,@lollipopdaisy I have It's jed
...,...,...,...,...,...,...
11284,0,1551291795,Sat Apr 18 08:39:57 PDT 2009,NO_QUERY,Srizzil,7-10 Crusaders Lead!
819732,1,1553492848,Sat Apr 18 14:24:55 PDT 2009,NO_QUERY,vaporcat,Allo internets. Out? Now? Dammit. &lt;3 i has...
813158,1,1548702733,Fri Apr 17 21:42:44 PDT 2009,NO_QUERY,Calilily,yummm the best tasting rice crispees ever! It ...
860,0,1468023146,Mon Apr 06 23:18:33 PDT 2009,NO_QUERY,EikLovesYou,@reannaremick doesnt work on my cell go to sl...


In [52]:
df_test

,target,ids,date,flag,user,text
808553,1,1469243500,Tue Apr 07 05:44:17 PDT 2009,NO_QUERY,einsteinradio,"@petshopboys Good stuff sirs, I hope it's soon"
9427,0,1548745312,Fri Apr 17 21:50:46 PDT 2009,NO_QUERY,Paolo1792,Just woke up.. It's 12:49pm here in the Philip...
199,0,1467861413,Mon Apr 06 22:32:58 PDT 2009,NO_QUERY,AmyJade,couldn't get shit done today ~ i'm so screwed
12447,0,1551807543,Sat Apr 18 10:01:29 PDT 2009,NO_QUERY,grimlok,this rick ross album makes me wish i had a car
814489,1,1550819919,Sat Apr 18 07:21:27 PDT 2009,NO_QUERY,lilellis92,nice day outside so think its the right time f...
...,...,...,...,...,...,...
803567,1,1468394654,Tue Apr 07 01:24:12 PDT 2009,NO_QUERY,MissBossiie,Being The SweetHeart That I Am!
800079,1,1467825491,Mon Apr 06 22:23:36 PDT 2009,NO_QUERY,Scyranth,@mattgalloway Thanks for the hook up with @Car...
18707,0,1556668828,Sat Apr 18 23:48:03 PDT 2009,NO_QUERY,firewontquell,ate an insane amount of food at @bombnomnom's ...
15200,0,1554038553,Sat Apr 18 15:54:03 PDT 2009,NO_QUERY,mandyyx,fucking a


In [53]:
print(Counter(df_train['target']))
print(Counter(df_test['target']))

Counter({1: 20022, 0: 19978})
Counter({0: 5022, 1: 4978})


In [54]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer

# Preprocess text
X_train_preprocessed = np.array([text_preprocessing(text) for text in df_train['text'].values])
X_test_preprocessed = np.array([text_preprocessing(text) for text in df_test['text'].values])

In [55]:
X_train_preprocessed

array(['teacup yorkie', 'friday till monday plan alot relaxing beach',
       'great read almost 1000 names less 45 minutes', ...,
       'yummm best tasting rice crispees ever marshmellows',
       'doesnt work cell go sleep p', 'dont understand cant pull'],
      dtype='<U150')

In [10]:
import pickle
with open('X_train_preprocessed.pickle', 'wb') as train:
    pickle.dump(X_train_preprocessed, train)
    
with open('X_test_preprocessed.pickle', 'wb') as test:
    pickle.dump(X_test_preprocessed, test)

In [25]:
import pickle
with open('X_train_preprocessed.pickle', 'rb') as train:
    X_train_preprocessed = pickle.load(train)
    
with open('X_test_preprocessed.pickle', 'rb') as test:
    X_test_preprocessed = pickle.load(test)

In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Calculate TF-IDF
tf_idf = TfidfVectorizer(ngram_range=(1, 3), smooth_idf=False)

X_train_tfidf = tf_idf.fit_transform(X_train_preprocessed)
X_test_tfidf = tf_idf.transform(X_test_preprocessed)

In [57]:
y_train = df_train['target'].values

In [58]:
len(X_train_preprocessed)

40000

In [59]:
len(y_train)

40000

In [60]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB().fit(X_train_tfidf, y_train)

In [62]:
from sklearn.metrics import  classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
labels_pred = nb.predict(X_test_tfidf)
#labels_pred = np.round(labels_pred.flatten())
y_test = df_test['target'].values
accuracy = accuracy_score(y_test, labels_pred)
f1 = f1_score(y_test, labels_pred, average='weighted')
precision = precision_score(y_test, labels_pred, average='weighted')
recall = recall_score(y_test, labels_pred, average='weighted')

print("Accuracy: %.2f%%" % (accuracy*100))
print("F1 Score: %.2f" % (f1*100))
print("Precision: %.2f" % (precision*100))
print("Recall: %.2f" % (recall*100))

print(classification_report(y_test, labels_pred))

Accuracy: 76.38%
F1 Score: 76.36
Precision: 76.45
Recall: 76.38
              precision    recall  f1-score   support

           0       0.75      0.79      0.77      5022
           1       0.78      0.74      0.76      4978

    accuracy                           0.76     10000
   macro avg       0.76      0.76      0.76     10000
weighted avg       0.76      0.76      0.76     10000



In [63]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, labels_pred)

array([[3979, 1043],
       [1319, 3659]])

In [64]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=42, max_iter=500).fit(X_train_tfidf, y_train)

In [66]:
from sklearn.metrics import  classification_report, confusion_matrix, accuracy_score, f1_score
y_test = df_test['target'].values
labels_pred2 = clf.predict(X_test_tfidf)
#labels_pred = np.round(labels_pred.flatten())
accuracy_2 = accuracy_score(y_test, labels_pred2)
f1_2 = f1_score(y_test, labels_pred2, average='weighted')
precision_2 = precision_score(y_test, labels_pred2, average='weighted')
recall_2 = recall_score(y_test, labels_pred2, average='weighted')

print("Accuracy: %.2f%%" % (accuracy_2*100))
print("F1 Score: %.2f" % (f1_2*100))
print("Precision: %.2f" % (precision_2*100))
print("Recall: %.2f" % (recall_2*100))

print(classification_report(y_test, labels_pred2))

Accuracy: 76.25%
F1 Score: 76.25
Precision: 76.25
Recall: 76.25
              precision    recall  f1-score   support

           0       0.76      0.77      0.76      5022
           1       0.76      0.76      0.76      4978

    accuracy                           0.76     10000
   macro avg       0.76      0.76      0.76     10000
weighted avg       0.76      0.76      0.76     10000



In [67]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, labels_pred2)

array([[3865, 1157],
       [1218, 3760]])

In [68]:
from sklearn.linear_model import SGDClassifier
clf_svm = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-6, random_state=42)
clf_svm.fit(X_train_tfidf, y_train)

SGDClassifier(alpha=1e-06, random_state=42)

In [75]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score

labels_pred4 = clf_svm.predict(X_test_tfidf)
y_test = df_test['target'].values
#labels_pred = np.round(labels_pred.flatten())
accuracy_4 = accuracy_score(y_test, labels_pred4)
f1_4 = f1_score(y_test, labels_pred4, average='weighted')
precision_4 = precision_score(y_test, labels_pred4, average='weighted')
recall_4 = recall_score(y_test, labels_pred4, average='weighted')

print("Accuracy: %.2f%%" % (accuracy_4*100))
print("F1 Score: %.2f" % (f1_4*100))
print("Precision: %.2f" % (precision_4*100))
print("Recall: %.2f" % (recall_4*100))

print(classification_report(y_test, labels_pred4))

Accuracy: 76.12%
F1 Score: 76.12
Precision: 76.12
Recall: 76.12
              precision    recall  f1-score   support

           0       0.76      0.77      0.76      5022
           1       0.76      0.75      0.76      4978

    accuracy                           0.76     10000
   macro avg       0.76      0.76      0.76     10000
weighted avg       0.76      0.76      0.76     10000



In [70]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, labels_pred4)

array([[3857, 1165],
       [1223, 3755]])

In [71]:
from sklearn.ensemble import RandomForestClassifier
randomforest = RandomForestClassifier(n_estimators = 100, random_state = 24)
randomforest.fit(X_train_tfidf, y_train)

RandomForestClassifier(random_state=24)

In [72]:
y_test = df_test['target'].values
labels_pred3 = randomforest.predict(X_test_tfidf)
#labels_pred = np.round(labels_pred.flatten())
accuracy_3 = accuracy_score(y_test, labels_pred3)
f1_3 = f1_score(y_test, labels_pred3, average='weighted')
precision_3 = precision_score(y_test, labels_pred3, average='weighted')
recall_3 = recall_score(y_test, labels_pred3, average='weighted')

print("Accuracy: %.2f%%" % (accuracy_3*100))
print("F1 Score: %.2f" % (f1_3*100))
print("Precision: %.2f" % (precision_3*100))
print("Recall: %.2f" % (recall_3*100))

print(classification_report(y_test, labels_pred3))

Accuracy: 74.83%
F1 Score: 74.82
Precision: 74.87
Recall: 74.83
              precision    recall  f1-score   support

           0       0.76      0.73      0.74      5022
           1       0.74      0.77      0.75      4978

    accuracy                           0.75     10000
   macro avg       0.75      0.75      0.75     10000
weighted avg       0.75      0.75      0.75     10000



In [3]:
import requests
 
url = "https://api.aiforthai.in.th/ssense"
 
text = 'I like this game'
 
params = {'text':text}
 
headers = {
    'Apikey': "g9DsotewMbQATsKL74HhB7jw1F16nEmm"
    }
 
response = requests.get(url, headers=headers, params=params)
 
print(response.json())

{'sentiment': {'score': '0', 'polarity-neg': False, 'polarity-pos': False, 'polarity': ''}, 'preprocess': {'input': 'I like this game', 'neg': [], 'pos': [], 'segmented': ['I', ' ', 'like', ' ', 'this', ' ', 'game'], 'keyword': ['I', 'like', 'this', 'game']}, 'alert': [], 'comparative': [], 'associative': [], 'intention': {'request': '0', 'sentiment': '0', 'question': '0', 'announcement': '0'}}


## Suicidal Data

In [3]:
suicide_df = pd.read_csv('../Dataset/Twitter_Suicide_Data_new.csv')
suicide_df

,Id,Content,Sentiment
0,4.964062e+07,The end.,Negative
1,4.350528e+07,GOD OVER EVERYTHING.,Negative
2,4.350528e+07,I'm sorry.,Negative
3,4.350528e+07,God... please forgive me.,Negative
4,4.350528e+07,This day couldn't get any worse...,Negative
...,...,...,...
298,9.334510e+17,"endless pain in life,end it",Negative
299,9.368910e+17,lets ave a lot of fun,Positive
300,9.123420e+17,"Nothing left in this world for me,i want to die",Negative
301,9.368910e+17,I am successful in life,Positive


In [4]:
y_train = suicide_df['Sentiment']
y_train_dict = Counter(y_train)
total = sum(y_train_dict.values())
print("Balanced Checking: ", Counter(y_train))
print("Label 0: %.2f%%" % (y_train_dict.get('Negative')*100 / total))
print("Label 1: %.2f%%" % (y_train_dict.get('Positive')*100 / total))

Balanced Checking:  Counter({'Negative': 175, 'Positive': 128})
Label 0: 57.76%
Label 1: 42.24%


In [27]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(suicide_df, test_size=0.2, random_state=42)

In [28]:
import nltk
# Uncomment to download "stopwords"
#nltk.download("stopwords")
from nltk.corpus import stopwords

def text_preprocessing(s):
    """
    - Lowercase the sentence
    - Change "'t" to "not"
    - Remove "@name"
    - Isolate and remove punctuations except "?"
    - Remove other special characters
    - Remove stop words except "not" and "can"
    - Remove trailing whitespace
    """
    s = s.lower()
    # Change 't to 'not'
    s = re.sub(r"\'t", " not", s)
    # Remove @name
    s = re.sub(r'(@.*?)[\s]', ' ', s)
    # Isolate and remove punctuations except '?'
    s = re.sub(r'([\'\"\.\(\)\!\?\\\/\,])', r' \1 ', s)
    s = re.sub(r'[^\w\s\?]', ' ', s)
    # Remove some special characters
    s = re.sub(r'([\;\:\|•«\n])', ' ', s)
    # Remove stopwords except 'not' and 'can'
    s = " ".join([word for word in s.split()
                  if word not in stopwords.words('english')
                  or word in ['not', 'can']])
    # Remove trailing whitespace
    s = re.sub(r'\s+', ' ', s).strip()
    
    return s

In [29]:
print(Counter(df_train['Sentiment']))
print(Counter(df_test['Sentiment']))

Counter({'Negative': 135, 'Positive': 107})
Counter({'Negative': 40, 'Positive': 21})


In [30]:
encoding = { 'Negative': 0,
    'Positive': 1,
}

y_train = [encoding[key] for key in df_train['Sentiment'].values]
y_test = [encoding[key] for key in df_test['Sentiment'].values]

In [31]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer

# Preprocess text
X_train_preprocessed = np.array([text_preprocessing(text) for text in df_train['Content'].values])
X_test_preprocessed = np.array([text_preprocessing(text) for text in df_test['Content'].values])

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Calculate TF-IDF
tf_idf = TfidfVectorizer(ngram_range=(1, 3), smooth_idf=False, max_features=100)

X_train_tfidf = tf_idf.fit_transform(X_train_preprocessed)
X_test_tfidf = tf_idf.transform(X_test_preprocessed)

In [10]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB().fit(X_train_tfidf, df_train['Sentiment'].values)

In [16]:
from sklearn.metrics import  classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
labels_pred = nb.predict(X_test_tfidf)
#labels_pred = np.round(labels_pred.flatten())
y_test = df_test['Sentiment'].values
accuracy = accuracy_score(y_test, labels_pred)
f1 = f1_score(y_test, labels_pred, average='weighted')
precision = precision_score(y_test, labels_pred, average='weighted')
recall = recall_score(y_test, labels_pred, average='weighted')

print("Accuracy: %.3f" % (accuracy))
print("F1 Score: %.3f" % (f1))
print("Precision: %.3f" % (precision))
print("Recall: %.3f" % (recall))

print(classification_report(y_test, labels_pred))

Accuracy: 0.951
F1 Score: 0.951
Precision: 0.952
Recall: 0.951
              precision    recall  f1-score   support

    Negative       0.97      0.95      0.96        40
    Positive       0.91      0.95      0.93        21

    accuracy                           0.95        61
   macro avg       0.94      0.95      0.95        61
weighted avg       0.95      0.95      0.95        61



In [17]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, labels_pred)

array([[38,  2],
       [ 1, 20]])

In [18]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=42).fit(X_train_tfidf, df_train['Sentiment'].values)

In [20]:
from sklearn.metrics import  classification_report, confusion_matrix, accuracy_score, f1_score

labels_pred2 = clf.predict(X_test_tfidf)
#labels_pred = np.round(labels_pred.flatten())
accuracy = accuracy_score(y_test, labels_pred2)
f1 = f1_score(y_test, labels_pred2, average='weighted')
precision = precision_score(y_test, labels_pred2, average='weighted')
recall = recall_score(y_test, labels_pred2, average='weighted')

print("Accuracy: %.3f" % (accuracy))
print("F1 Score: %.3f" % (f1))
print("Precision: %.3f" % (precision))
print("Recall: %.3f" % (recall))

print(classification_report(y_test, labels_pred2))

Accuracy: 0.967
F1 Score: 0.967
Precision: 0.967
Recall: 0.967
              precision    recall  f1-score   support

    Negative       0.97      0.97      0.97        40
    Positive       0.95      0.95      0.95        21

    accuracy                           0.97        61
   macro avg       0.96      0.96      0.96        61
weighted avg       0.97      0.97      0.97        61



In [136]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, labels_pred2)

array([[39,  1],
       [ 1, 20]])

In [33]:
from sklearn.ensemble import RandomForestClassifier
randomforest = RandomForestClassifier(n_estimators = 100, random_state = 24)
randomforest.fit(X_train_tfidf, df_train['Sentiment'].values)

RandomForestClassifier(random_state=24)

In [35]:
labels_pred3 = randomforest.predict(X_test_tfidf)
#labels_pred = np.round(labels_pred.flatten())
y_test = df_test['Sentiment'].values
accuracy_3 = accuracy_score(y_test, labels_pred3)
f1_3 = f1_score(y_test, labels_pred3, average='weighted')
precision_3 = precision_score(y_test, labels_pred3, average='weighted')
recall_3 = recall_score(y_test, labels_pred3, average='weighted')

print("Accuracy: %.2f%%" % (accuracy_3*100))
print("F1 Score: %.2f" % (f1_3*100))
print("Precision: %.3f" % (precision_3))
print("Recall: %.3f" % (recall_3))

print(classification_report(y_test, labels_pred3))

Accuracy: 93.44%
F1 Score: 93.51
Precision: 0.938
Recall: 0.934
              precision    recall  f1-score   support

    Negative       0.97      0.93      0.95        40
    Positive       0.87      0.95      0.91        21

    accuracy                           0.93        61
   macro avg       0.92      0.94      0.93        61
weighted avg       0.94      0.93      0.94        61



In [139]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, labels_pred3)

array([[35,  5],
       [ 3, 18]])

In [24]:
from sklearn.svm import SVC
SVM = SVC(kernel='linear', random_state=42).fit(X_train_tfidf, df_train['Sentiment'].values)

In [25]:
labels_pred4 = SVM.predict(X_test_tfidf)
#labels_pred = np.round(labels_pred.flatten())
accuracy_4 = accuracy_score(y_test, labels_pred4)
f1_4 = f1_score(y_test, labels_pred4, average='weighted')
precision_4 = precision_score(y_test, labels_pred4, average='weighted')
recall_4 = recall_score(y_test, labels_pred4, average='weighted')

print("Accuracy: %.2f%%" % (accuracy_4*100))
print("F1 Score: %.2f" % (f1_4*100))
print("Precision: %.3f" % (precision_4))
print("Recall: %.3f" % (recall_4))

print(classification_report(y_test, labels_pred4))

Accuracy: 95.08%
F1 Score: 95.11
Precision: 0.952
Recall: 0.951
              precision    recall  f1-score   support

    Negative       0.97      0.95      0.96        40
    Positive       0.91      0.95      0.93        21

    accuracy                           0.95        61
   macro avg       0.94      0.95      0.95        61
weighted avg       0.95      0.95      0.95        61



In [37]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, labels_pred4)

array([[37,  3],
       [ 1, 20]])